In [223]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, dash_table, Input, Output
import warnings
from scipy import stats
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [224]:
# Load dataset (Replace with actual data source)
df = pd.read_csv('marketing_campaign.csv', sep=';')

In [225]:
# create some group of categories based on values
# customers by number of  kids at home
df['CustNumKidhome'] = df['Kidhome'].apply(lambda x: 'low number kids'if x <  df['Kidhome'].quantile(0.25)\
                                                                      else 'medium number kids' if x < df['Kidhome'].quantile(0.75)\
                                                                      else 'high number kids')
#customers by number of teenagers at home
df['CustTeenHome'] = df['Teenhome'].apply(lambda x: 'low number Teenagers'if x <=  df['Teenhome'].quantile(0.25)
                                                                      else 'medium number Teenagers' if x<df['Teenhome'].quantile(0.75)
                                                                      else 'high number Teenagers')
#customers by total amount of spending
df['TotalMnt'] = df[['MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

df['CustTotalMnt'] = df.TotalMnt.apply(lambda x: 'low spending'if x <  df['TotalMnt'].quantile(0.25)
                                                                      else 'medium spending' if x<df['TotalMnt'].quantile(0.75)
                                                                      else 'high spending')

#customers by incoming
df['CustIncome'] = df['Income'].apply(lambda x: 'low incomes'if x <=  df['Income'].quantile(0.25)
                                                                      else 'medium incomes' if x<df['Income'].quantile(0.75)
                                                                      else 'high incomes')
#response to one of the previous marketing campaign
df['RespondedToPrevious'] = df[[f'AcceptedCmp{i}' for i in range(1, 6)]].sum(axis=1)


In [235]:
df_summary = pd.DataFrame({
    "Columns": df.columns,
    "Type": df.dtypes.astype(str).values,
    "Missing values": df.isna().sum().values
})
numeric_features = ['Income', 'Recency', 'MntWines', 'MntFruits','MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
                                                           'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
                                                           'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
                                                           'TotalMnt','RespondedToPrevious']
categorical_features = ['Education', 'Marital_Status', 'Kidhome','Teenhome', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
                       'AcceptedCmp2', 'Complain', 'Response',
                       'CustNumKidhome', 'CustTeenHome', 'CustTotalMnt',
                       'CustIncome', 'RespondedToPrevious']
corr_heatmap_features = numeric_features + ['Kidhome','Teenhome', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
                       'AcceptedCmp2', 'Complain', 'Response']

In [240]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Customer Analytics Dashboard"),

     # Summary & KPIs
    html.H3("Summary & KPIs"),
    html.Div([
        html.P(f"Total Customers: {len(df)}"),
        html.P(f"Response Rate by the last campaign: {df['Response'].mean():.2f}%"),
        html.P(f"Average Income: {df['Income'].mean():.2f}$"),
    ]),

    html.H3("Quality of Customer's data 📊"),
    dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in df_summary.columns],
        data=df_summary.to_dict("records"),
        page_size=5
    ),

    # Campaign Performance Analysis
    html.H3("Campaign Performance Analysis"),
    dcc.Graph(id="campaign-bar-chart"),
    html.Br(),
    html.H3("Multivariate  Analysis"),
    dcc.Dropdown(
        id="categorical-variable",
        options=[{"label": col, "value": col} for col in categorical_features],
        value=categorical_features[0],
    ),
    dcc.Dropdown(
        id="numeric-variable",
        options=[{"label": col, "value": col} for col in numeric_features],
        value= numeric_features[0]
    ),
    dcc.Graph(id="box-plot-chart"),

    html.Br(),

    dcc.Dropdown(
        id="x-variable",
        options=[{"label": col, "value": col} for col in numeric_features],
        value=numeric_features[0]
    ),
    dcc.Dropdown(
        id="y-variable",
        options=[{"label": col, "value": col} for col in numeric_features],
        value=numeric_features[1]
    ),
    dcc.Graph(id="regression-plot"),
    html.Br(),

    html.H3("Correlation between features 📊"),
    dcc.Checklist(
         id="variable",
        options=[{"label": col, "value": col} for col in corr_heatmap_features],
        value=[corr_heatmap_features[0], corr_heatmap_features[1], corr_heatmap_features[2]]
    ),
    dcc.Graph(id='correlation-heatmap'),

    html.Br(),

    html.H3("Test Chi² 🔢"),
    dcc.Dropdown(
        id="chi2-var1",
        options=[{"label": col, "value": col} for col in categorical_features],
        value=categorical_features[0],
    ),
    dcc.Dropdown(
        id="chi2-var2",
        options=[{"label": col, "value": col} for col in categorical_features],
        value= categorical_features[1]
    ),
    dcc.Graph(id="chi2-heatmap"),
    html.Div(id="chi2-result"),

    html.Br(),

    html.H3("select a Feature for the t-Test"),
    dcc.Dropdown(
        id="t-test-feature",
        options=[{"label": col, "value": col} for col in ['NumDealsPurchases', 'NumWebPurchases',
                                                         'NumCatalogPurchases', 'NumStorePurchases','NumWebVisitsMonth']],
        value= 'NumDealsPurchases'
    ),

   html.H3("select variables the selected Feature"),
   dcc.Dropdown(
        id="t-test-variable",
        options=[{"label": col, "value": col} for col in ['TotalMnt', 'RespondedToPrevious', 'Response', 'MntWines', 'MntFruits',
                                                            'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','MntGoldProds']],
        value='TotalMnt',
    ),
    dcc.Graph(id="t-test-boxplot"),
    html.Div(id="t-test-result"),

    html.Br(),

    html.H3("select a categorical variable for the Test ANOVA 📊"),
    dcc.Dropdown(
        id="anova-test-feature",
        options=[{"label": col, "value": col} for col in categorical_features],
        value= categorical_features[0],
    ),

   html.H3("select numerical variables for the selected feature"),
    dcc.Dropdown(
        id="anova-test-variable",
        options=[{"label": col, "value": col} for col in numeric_features + ['Kidhome','Teenhome',
                                                                             'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
                                                                             'AcceptedCmp2', 'Complain', 'Response']],
        value=numeric_features[0],
    ),
    dcc.Graph(id="anova-test-boxplot"),
    html.Div(id="anova-test-result"),
])

# Callbacks
@app.callback(
    Output("campaign-bar-chart", "figure"),
    Input("campaign-bar-chart", "id")
)
def update_campaign_chart(_):
    x = [f'AcceptedCmp{i}' for i in range(1,6)]
    y = [df[col].mean() for col in x] + [df['Response'].mean()]
    label = [f'accepted response rate by campaign {i}' for i in range(1,6)] + ['response positiv by last campaign']
    fig = px.bar(x=label, y=y, title="Campaign Acceptance Rates")
    return fig

# Callbacks
@app.callback(
    Output("box-plot-chart", "figure"),
    [Input("categorical-variable", "value"), Input("numeric-variable", "value")]
)
def update_box_chart(x_label, y_label):
    fig = px.box(df, x=x_label, y=y_label , title=f"box chart plot of {x_label} vs {y_label}")
    return fig

@app.callback(
    Output("regression-plot", "figure"),
    [Input("x-variable", "value"), Input("y-variable", "value")]
)
def update_regression(x_var, y_var):
    fig = px.scatter(df, x=x_var, y=y_var, trendline="ols", title=f"Linear Regression : {x_var} vs {y_var}")
    return fig

@app.callback(
    Output("correlation-heatmap", "figure"),
    Input("variable", "value")
)
def update_correlation(values):
    corr_matrix = df[values].corr()
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale="RdBu",
        text=corr_matrix.values,
        texttemplate="%{text:.2f}"
    ))
    fig.update_layout(title="correlation heatmap")
    return fig

@app.callback(
    [Output("chi2-heatmap", "figure"), Output("chi2-result", "children")],
    [Input("chi2-var1", "value"), Input("chi2-var2", "value")]
)
def update_chi2_test(var1, var2):
    contingency_table = pd.crosstab(df[var1], df[var2])
    chi2_stat, p, dof, expected = stats.chi2_contingency(contingency_table)

    fig = go.Figure(data=go.Heatmap(
        z=expected,
        x=contingency_table.columns,
        y=contingency_table.index,
        colorscale="Blues",
        texttemplate="%{z:.2f}"
    ))

    fig.update_layout(title=f"Table of excepted frequency - {var1} vs {var2}")

    result_text = f"Test Chi²: statistic = {chi2_stat:.2f}, p-value = {p:.4f}"

    return fig, html.P(result_text, style={"font-weight": "bold", "color": "blue"})

@app.callback(
    [Output("t-test-boxplot", "figure"), Output("t-test-result", "children")],
    [Input("t-test-variable", "value"), Input("t-test-feature", "value")]
)
#Do customers who purchase through discounts (NumDealsPurchases) have different spending behaviors compared to those who don’t?
def update_t_test(variable, feature):
    group_A = df[df[feature]> 1][variable]
    group_B = df[df[feature] == 0][variable]

    t_stat, p_value = stats.ttest_ind(group_A, group_B)

    # create box plot

    fig = px.pie(values=[group_A.sum(), group_B.sum()],
                 names=[f'customers who purchase through {feature}', f'customers who doesnt purchase through {feature}'] )

    result_text = f"Test t : Statistique = {t_stat:.2f}, p-value = {p_value:.4f}"

    return fig, html.P(result_text, style={"font-weight": "bold", "color": "blue"})

@app.callback(
    [Output("anova-test-boxplot", "figure"), Output("anova-test-result", "children")],
    [Input("anova-test-variable", "value"), Input("anova-test-feature", "value")]
)
def update_anova_test(variable, feature):
    groups = [df[df[feature] == g][variable] for g in df[feature].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    fig = px.pie(values=[g.sum() for g in groups],names=[f for f in df[feature].unique()])

    result_text = f"Test ANOVA: F = {f_stat:.2f}, p-value = {p_value:.4f}"

    return fig, html.P(result_text, style={"font-weight": "bold", "color": "blue"})

if __name__ == "__main__":
    app.run_server(debug=True, port=8051)
